In [7]:
from packages import constants
import duckdb
import plotly.express as px

In [8]:
conn = duckdb.connect(constants.DATABASE_NAME)

In [9]:
# Creating Transformed Users Table
query = """
DROP TABLE IF EXISTS USERS_TRANSFORMED;

CREATE TABLE USERS_TRANSFORMED AS
SELECT
userid,
created_at,
MONTH(created_at)::INTEGER AS MONTH_CREATED,
YEAR(created_at)::INTEGER AS YEAR_CREATED,
DAY(created_at)::INTEGER AS DAY_CREATED,
HOUR(created_at)::INTEGER AS HOUR_CREATED,
FROM users;
"""
conn.execute(query)

In [11]:
# Creating Transformed Loggin Table
query = """
DROP TABLE IF EXISTS LOGON_EVENTS_TRANSFORMED;

CREATE TABLE LOGON_EVENTS_TRANSFORMED AS
SELECT
userid,
logon_ts,
MONTH(logon_ts)::INTEGER AS MONTH_LOGON,
YEAR(logon_ts)::INTEGER AS YEAR_LOGON,
DAY(logon_ts)::INTEGER AS DAY_LOGON,
HOUR(logon_ts)::INTEGER AS HOUR_LOGON,
FROM logon_events;
"""
conn.execute(query)

In [12]:
df_users_transformed = conn.execute('FROM USERS_TRANSFORMED').df()
df_logon_events_transformed = conn.execute('FROM LOGON_EVENTS_TRANSFORMED').df()

In [13]:
print(df_users_transformed.columns)
print(df_logon_events_transformed.columns)

Index(['userid', 'created_at', 'MONTH_CREATED', 'YEAR_CREATED', 'DAY_CREATED',
       'HOUR_CREATED'],
      dtype='object')
Index(['userid', 'logon_ts', 'MONTH_LOGON', 'YEAR_LOGON', 'DAY_LOGON',
       'HOUR_LOGON'],
      dtype='object')


In [14]:
df_users_transformed.head()

,userid,created_at,MONTH_CREATED,YEAR_CREATED,DAY_CREATED,HOUR_CREATED
0,user1,2023-12-08 23:18:37.722,12,2023,8,23
1,user2,2023-11-27 17:21:01.722,11,2023,27,17
2,user3,2023-12-17 11:08:53.722,12,2023,17,11
3,user4,2023-12-05 11:27:57.722,12,2023,5,11
4,user5,2023-12-07 16:45:56.722,12,2023,7,16


In [18]:
fig = px.histogram(df_users_transformed, x='HOUR_CREATED',)
fig.update_layout(
    title='Hours Created Accounts',
    width=400,
    height=300,
    xaxis_title='Total Hours',
    yaxis_title='Hours Created',
    template='simple_white',
)

In [19]:
fig = px.histogram(df_logon_events_transformed, x='HOUR_LOGON',)
fig.update_layout(
    title='Hours Logon Accounts',
    width=400,
    height=300,
    xaxis_title='Total Hours',
    yaxis_title='Hours Logon',
    template='simple_white',
)

In [46]:
query = """
SELECT
    u.userid,
    SUM(CASE WHEN date_diff('day', u.created_at, le.logon_ts) = 1 THEN 1 ELSE 0 END) AS one_date_diff,
    SUM(CASE WHEN date_diff('day', u.created_at, le.logon_ts) = 2 THEN 1 ELSE 0 END) AS two_date_diff,
    SUM(CASE WHEN date_diff('day', u.created_at, le.logon_ts) = 3 THEN 1 ELSE 0 END) AS three_date_diff,
FROM users u
LEFT JOIN logon_events le ON u.userid = le.userid
GROUP BY
    u.userid
ORDER BY
    u.userid
"""
df_date_dif = conn.execute(query).df()

In [53]:
fig = px.histogram(df_date_dif, x='one_date_diff', width=500, height=300)
fig.show()

In [55]:
query = """
SELECT
    u.userid,
    u.created_at,
    le.logon_ts,
    date_diff('day', u.created_at, le.logon_ts) AS date_diff,
    date_diff('hour', u.created_at, le.logon_ts) AS hour_diff,
FROM users u
LEFT JOIN logon_events le ON u.userid = le.userid
WHERE
    le.logon_ts IS NOT NULL
"""
df_date_diff = conn.execute(query).df()

In [56]:
df_date_diff

,userid,created_at,logon_ts,date_diff,hour_diff
0,user1,2023-12-08 23:18:37.722,2023-12-11 23:18:37.722,3,72
1,user2,2023-11-27 17:21:01.722,2023-11-28 17:21:01.722,1,24
2,user3,2023-12-17 11:08:53.722,2023-12-18 11:08:53.722,1,24
3,user4,2023-12-05 11:27:57.722,2023-12-07 14:27:57.722,2,51
4,user5,2023-12-07 16:45:56.722,2023-12-10 16:45:56.722,3,72
5,user6,2023-12-17 15:36:16.722,2023-12-20 15:36:16.722,3,72
6,user9,2023-12-02 01:11:58.722,2023-12-05 01:11:58.722,3,72
7,user10,2023-12-15 20:06:11.722,2023-12-18 20:06:11.722,3,72
8,user1,2023-12-08 23:18:37.722,2023-12-10 23:18:37.722,2,48
9,user4,2023-12-05 11:27:57.722,2023-12-07 11:27:57.722,2,48


In [60]:
fig = px.histogram(df_date_diff, x='hour_diff')
fig.show()